# Programming in Python


## Getting to know Python

### Different ways to interact with Python
 1. Using the interpreter (for instance `ipython`)
 2. Using the `jupyter notebook`
 3. Writing and executing programs (`python myProgram.py`)

### Python language elements

 * Python as a calculator 
   * operations 
   * math functions

 * strings, numbers, lists and dictionaries
   * short and long strings
     * concatenating 
     * converting numbers to a string 
   * integer and floating point numbers
     * integers can grow
     * converting a string to a number
   * packing a bunch of things together with a **list**
     * length, inserting, appending, sorting and combining
   * accessing values by keys with a **dictionary**
     * key must unique

 * indexing 

 * storing data in **variables**

 * printing

 * **Identation** is part of the language!
   * loops
   * conditionals
   * scoping

* functions

 * comments

### Common pitfalls
 * watch out for indentation
 * comparing floats may yield unintented consequences
 * integer division (//) vs division (/)
 * in place vs function that return new objects

## Python with batteries charged

### Plotting with matplotlib 

 * Line plots
 * Surface plots
 * Plotting inside `jupyter` documents

### Numpy for fast array operations (Matlab and IDL users pay attention)

$$\frac{\partial f}{\partial t} + u \frac{\partial f}{\partial x} = 0$$
$$f_i^{t + \Delta t} = (1 - C) f_i^t + C f_{i-1}^t$$ with $$C \equiv \frac{u \Delta t}{\Delta x}$$

### Pandas for data analysis (R users pay attention)

### Growing from here -- Installing additional packages